### EDS 296: Assignment 1

Author: Liz Peterson
Date: 4-21-2025

In [3]:
# Import necessary packages
import xarray as xr
import matplotlib.pyplot as plt
import intake
import s3fs

In [4]:
# Open the CMIP6 data catalog & store as a variable
catalog = intake.open_esm_datastore('https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.json')

I've chosen to look at heatwaves and water scarcity in the Southwestern US. To investigate this, I am using the `CanESM5` and `GFDL-ESM4` models. I want to look at surface air temperature as a proxy for investigating heatwaves. This is a very hot and arid place. Heat waves and rising surface temperatures are a threat that can lead to water scarcity. 

In [11]:
# Specify search terms to query catalog for my two models 
# activity_id: which project do you want? CMIP = historical data, ScenarioMIP = future projections
activity_ids = ['ScenarioMIP', 'CMIP']

# source_id: which model do you want? 
source_id = ['CanESM5', 'GFDL-ESM4']

# experiment_id: what experimental configuration do you want? Here we want historical and the basic 4 scenarios
experiment_ids = ['historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585']

# member_id: which ensemble member do you want? Here we want r10i1p1f1
member_id = 'r1i1p1f1'

# table_id: which part of the Earth system and time resolution do you want? Here we want monthly atmosphere data
table_id = 'Amon'

# variable_id: which climate variable do you want? Here we want surface air temperature
variable_id = 'tas'

In [12]:
# Search through catalog, store results in "res" variable
res = catalog.search(activity_id=activity_ids, source_id=source_id, experiment_id=experiment_ids, 
                     member_id=member_id, table_id=table_id, variable_id=variable_id)

# Display data frame associated with results
display(res.df)

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,ScenarioMIP,NOAA-GFDL,GFDL-ESM4,ssp245,r1i1p1f1,Amon,tas,gr1,s3://cmip6-pds/CMIP6/ScenarioMIP/NOAA-GFDL/GFD...,NaN,20180701
1,ScenarioMIP,NOAA-GFDL,GFDL-ESM4,ssp370,r1i1p1f1,Amon,tas,gr1,s3://cmip6-pds/CMIP6/ScenarioMIP/NOAA-GFDL/GFD...,NaN,20180701
2,ScenarioMIP,NOAA-GFDL,GFDL-ESM4,ssp126,r1i1p1f1,Amon,tas,gr1,s3://cmip6-pds/CMIP6/ScenarioMIP/NOAA-GFDL/GFD...,NaN,20180701
3,ScenarioMIP,NOAA-GFDL,GFDL-ESM4,ssp585,r1i1p1f1,Amon,tas,gr1,s3://cmip6-pds/CMIP6/ScenarioMIP/NOAA-GFDL/GFD...,NaN,20180701
4,ScenarioMIP,CCCma,CanESM5,ssp585,r1i1p1f1,Amon,tas,gn,s3://cmip6-pds/CMIP6/ScenarioMIP/CCCma/CanESM5...,NaN,20190429
5,ScenarioMIP,CCCma,CanESM5,ssp370,r1i1p1f1,Amon,tas,gn,s3://cmip6-pds/CMIP6/ScenarioMIP/CCCma/CanESM5...,NaN,20190429
6,CMIP,CCCma,CanESM5,historical,r1i1p1f1,Amon,tas,gn,s3://cmip6-pds/CMIP6/CMIP/CCCma/CanESM5/histor...,NaN,20190429
7,ScenarioMIP,CCCma,CanESM5,ssp126,r1i1p1f1,Amon,tas,gn,s3://cmip6-pds/CMIP6/ScenarioMIP/CCCma/CanESM5...,NaN,20190429
8,ScenarioMIP,CCCma,CanESM5,ssp245,r1i1p1f1,Amon,tas,gn,s3://cmip6-pds/CMIP6/ScenarioMIP/CCCma/CanESM5...,NaN,20190429
9,CMIP,NOAA-GFDL,GFDL-ESM4,historical,r1i1p1f1,Amon,tas,gr1,s3://cmip6-pds/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/...,NaN,20190726


Now that I have my scenarios saved in my df, I want to bring together my historical and projection data. I am going to use ssp245 for a middle of the road projection and ssp370 for a high emissions scenario. First, I'm going to read in the historical and projection data for my two models, then I'll combine them. 

In [14]:
# Extract data for the historical period for CanESM5
hist_can = xr.open_zarr(res.df['zstore'][6], storage_options={'anon': True})

# Extract data for the historical period for GFDL-ESM4
hist_gfdl = xr.open_zarr(res.df['zstore'][9], storage_options={'anon': True})

# Extract data for ssp245 for CanESM5
ssp245_can = xr.open_zarr(res.df['zstore'][8], storage_options={'anon': True})

# Extract data for ssp370 for CanESM5
ssp370_can = xr.open_zarr(res.df['zstore'][5], storage_options={'anon': True})

# Extract data for ssp245 for GFDL-ESM4
ssp245_gfdl = xr.open_zarr(res.df['zstore'][0], storage_options={'anon': True})

# Extract data for ssp370 for GFDL-ESM4
ssp370_gfdl = xr.open_zarr(res.df['zstore'][1], storage_options={'anon': True})

In [15]:
# Concatenate historical and future projection data - SSP245 CanE5M5
can_245 = xr.concat([hist_can, ssp245_can], dim="time")

# Concatenate historical and future projection data - SSP370 CanESM5
can_370 = xr.concat([hist_can, ssp370_can], dim="time")

# Concatenate historical and future projection data - SSP245 GFDL-ESM4
gfdl_245 = xr.concat([hist_gfdl, ssp245_gfdl], dim="time")

# Concatenate historical and future projection data - SSP370 GFDL-ESM4
gfdl_370 = xr.concat([hist_gfdl, ssp370_gfdl], dim="time")

Now that I have all of my data, I want to revist the spatial aspect of the assignment. I am going to define my bounding box around the Southwestern US and then plot from there! Specifically, this bounding box defines the region around Arizona, Nevada, California, Utah, and New Mexico. I'm going to average the lat and lon for each model combination.

Let's start with the CanESM5 Model SSP 245 Projection

In [16]:
# Define min/max bounds for region of interest 
lat_min, lat_max = 31, 37
lon_min, lon_max = 245, 257

# Define logical mask for can
tas_can245_lat = (can_245.lat >= lat_min) & (can_245.lat <= lat_max)
tas_can245_lon = (can_245.lon >= lon_min) & (can_245.lon <= lon_max)

# Find points where the mask value is True, drop all other points
tas_can245 = can_245.where(tas_can245_lat & tas_can245_lon, drop=True)

# Average over lat, lon dimensions to get a time series
tas_can245 = tas_can245.mean(dim=["lat", "lon"])

CanESM5 Model SSP 370 Projection

In [ ]:
# Define min/max bounds for region of interest 
lat_min, lat_max = 31, 37
lon_min, lon_max = 245, 257

# Define logical mask for can
tas_can370_lat = (can_370.lat >= lat_min) & (can_370.lat <= lat_max)
tas_can370_lon = (can_370.lon >= lon_min) & (can_370.lon <= lon_max)

# Find points where the mask value is True, drop all other points
tas_can370 = can_370.where(tas_can370_lat & tas_can370_lon, drop=True)

# Average over lat, lon dimensions to get a time series
tas_can370 = tas_can370.mean(dim=["lat", "lon"])

GFDL-ESM4 Model SSP 245 Projection

In [ ]:
# Define min/max bounds for region of interest 
lat_min, lat_max = 31, 37
lon_min, lon_max = 245, 257

# Define logical mask for can
tas_gfdl_245_lat = (gfdl_245.lat >= lat_min) & (gfdl_245.lat <= lat_max)
tas_gfdl_245_lon = (gfdl_245.lon >= lon_min) & (gfdl_245.lon <= lon_max)

# Find points where the mask value is True, drop all other points
tas_gfdl245 = gfdl_245.where(tas_gfdl_245_lat & tas_gfdl_245_lon, drop=True)

# Average over lat, lon dimensions to get a time series
tas_gfdl245 = gfdl_245.mean(dim=["lat", "lon"])

GFDL-ESM4 Model SSP 370 Projection

In [ ]:
# Define min/max bounds for region of interest 
lat_min, lat_max = 31, 37
lon_min, lon_max = 245, 257

# Define logical mask for can
tas_gfdl370_lat = (gfdl_370.lat >= lat_min) & (gfdl_370.lat <= lat_max)
tas_gfdl370_lon = (gfdl_370.lon >= lon_min) & (gfdl_370.lon <= lon_max)

# Find points where the mask value is True, drop all other points
tas_gfdl370 = gfdl_370.where(tas_gfdl370_lat & tas_gfdl370_lon, drop=True)

# Average over lat, lon dimensions to get a time series
tas_gfdl370 = tas_gfdl370.mean(dim=["lat", "lon"])